# Functionalization

### Import

In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.init

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings(action='ignore') 

### Device setting

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Data setting

In [3]:
train_set = torchvision.datasets.MNIST(root = './content',
                                      train = True,
                                      download = False,
                                      transform=transforms.ToTensor())

test_set = torchvision.datasets.MNIST(root = './content',
                                     train = False,
                                     download = False,
                                     transform=transforms.ToTensor())

### Data size check

In [4]:
train_set.data.shape, test_set.data.shape

(torch.Size([60000, 28, 28]), torch.Size([10000, 28, 28]))

### Data Load

In [5]:
train_load = DataLoader(train_set, shuffle=True, 
                        batch_size=128,num_workers=3)

### My Modeling


In [6]:
class MyM(nn.Module):
    def __init__(self):
        super(MyM,self).__init__()
        
        self.call = nn.Conv2d(1,32,3,padding=1)
        nn.init.kaiming_uniform_(self.call.weight)
        self.batch_call = nn.BatchNorm2d(32)
        
        self.cnn1 = nn.Conv2d(1,128,3,padding=1)
        nn.init.kaiming_uniform_(self.cnn1.weight)
        self.batch1 = nn.BatchNorm2d(128)
        self.Relu1 = nn.ReLU()
        self.cnn2 = nn.Conv2d(128,64,3,padding=1)
        self.batch2 = nn.BatchNorm2d(64)
        self.Relu2 = nn.ReLU()
        self.cnn3 = nn.Conv2d(64,32,3,padding=1)
        self.batch3 = nn.BatchNorm2d(32)
        
        self.avgpooling = nn.AvgPool2d(2)
        
        self.linear1 = nn.Linear(32*14*14, 128)
        self.li_batch = nn.BatchNorm1d(128)
        self.li_Relu = nn.ReLU()
        self.linear2 = nn.Linear(128,10)
        
    def forward(self, x):
        
        call_out = self.call(x)
        call_out = self.batch_call(call_out)
        
        out = self.cnn1(x)
        out = self.batch1(out)
        out = self.Relu1(out)
        out = self.cnn2(out)
        out = self.batch2(out)
        out = self.Relu2(out)
        out = self.cnn3(out)
        out = self.batch3(out)
        
        result_out = call_out + out
        
        result_out = self.avgpooling(result_out)
        
        result_out = result_out.view(result_out.size(0),-1)
        
        result_out = self.linear1(result_out)
        result_out = self.li_batch(result_out)
        result_out = self.li_Relu(result_out)
        result_out = self.linear2(result_out)
        
        return result_out

### Model define

In [7]:
MYM = MyM()

### Functionalization

In [8]:
def Functionalization_madel(train_load,model,base_model='CNN',epochs=2,lr=0.001):

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    optimizer = optim.Adam(model.parameters(),lr=lr)
    loss_function = nn.CrossEntropyLoss().to(device)
    
    for i in range(epochs):
        check_loss = 0.0
        
        for j in train_load:
            image, labels = j
            
            if base_model == 'CNN':
                image = image.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                hypothesis = model(image)
                
                loss = loss_function(hypothesis, labels)
                
                loss.backward()
                
                optimizer.step()
                
            elif base_model == 'LINEAR':
                image = image.to(device)
                labels = labels.to(device)
                
                image = image.view(image.size(0),-1)
                
                optimizer.zero_grad()
                
                hypothesis = model(image)
                
                loss = loss_function(hypothesis, labels)
                
                loss.backward()
                
                optimizer.step()
            
            check_loss += loss / len(train_load)
            
        print(i,check_loss)
    print('finish')

In [9]:
Functionalization_madel(train_load,MYM,base_model='CNN')

0 tensor(0.1287, grad_fn=<AddBackward0>)
1 tensor(0.0410, grad_fn=<AddBackward0>)
finish


### Test performance check

In [10]:
def test_check(model, test_set, base_model = 'CNN'):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    with torch.no_grad():
        if base_model == 'CNN':
            x_test = test_set.data.view(len(test_set),1,28,28).float().to(device)
            y_test = test_set.test_labels.to(device)
            
            pred = model(x_test)
            check_pred = torch.argmax(pred,dim=1) == y_test
            acc = check_pred.float().mean()
            
        elif base_model == 'LINEAR':
            x_test = test_set.data.view(len(test_set),1,28,28).float().to(device)
            x_test = x_test.view(x_test.size(0),-1)
            y_test = test_set.test_labels.to(device)
            
            pred = model(x_test)
            check_pred = torch.argmax(pred,dim=1) == y_test
            acc = check_pred.float().mean()
            
    return acc

In [11]:
test_check(MYM,test_set,base_model='CNN')

tensor(0.9912)